# Binaural synthesis

First, we will briefly talk about the basics of binaural synthesis.

Humans can localize sound events about their perceived distance and their angular position in space.
Sound waves are altered due to reflections, diffraction, and resonances caused by the presence of a human body, head, shoulders, torso, and the fine structure of the ear formed by pinna, cavum conchae, etc.
All these effects, which in its assembly are evaluated by the human brain to localize a source or to get other spatial information, are integrated into binaural signals.
If the binaural signal is reproduced perfectly at the eardrums (the human microphones), there is no chance to distinguish the virtual source or environment from the real sound field.
With binaural synthesis, a filtering approach with special filters, an acoustic sound source represented by a mono-signal can be virtually placed at arbitrary space positions.

## HRTF dataset

A valid way to describe all linear sound transformations caused by torso, head, and pinna is using so-called “head-related transfer functions” (HRTFs).
For each direction of sound incidence from a sound source to a human receiver, two transfer functions exist (one for the left and one for the right ear), combined into a two-channel HRTF in the frequency domain.
Combining all directions into a single database is commonly called an HRTF dataset.
The time domain version of HRTFs are also known as "head-related impulse responses" (HRIRs).

In [ ]:
%matplotlib inline
# import the required packages
import pyfar as pf
import sofar as sf
import numpy as np
import matplotlib.pyplot as plt
import os

files = pf.signals.files._load_files('head_related_impulse_responses')
hrir_file = os.path.join(pf.signals.files.file_dir, files[0])

# load a SOFA file containing the HRIR dataset
hrirs, sources, _ = pf.io.read_sofa(hrir_file)


Here we are loading the included HRIR dataset from the FABIAN dummy head by [Brinkmann _et al._](https://depositonce.tu-berlin.de/items/3b423df7-a764-4ce1-9065-4e6034bba759).
`pyfar` includes a [method to load specific HRIRs from the dataset](https://pyfar.readthedocs.io/en/stable/modules/pyfar.signals.files.html#pyfar.signals.files.head_related_impulse_responses) but the example shown here is the general approach for loading a SOFA file.

First, we will plot all possible source locations that are contained in the dataset.

In [ ]:
sources.show()

We can see that the included dataset only contains a limited number of source positions - namely, the horizontal plane and the median plane.
In practice, an HRIR dataset usually contains more source positions covering the whole sphere around the listener.

Using the `pyfar` coordinates, <!-- LINK TODO --> a specific direction can be selected from the dataset.

In [ ]:
# define the direction of the desired sound source; here, we choose the right direction
elevation = 0
azimuth = -90

# convert to radians
elevation = elevation * np.pi / 180
azimuth = azimuth * np.pi / 180

desired_direction = pf.Coordinates.from_spherical_elevation(
    azimuth, elevation, 2
)

index, _ = sources.find_nearest(desired_direction)

sources.show(index)

In the plot, the selected direction is marked with a red dot.